In [ ]:
# import the libraries 
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, GroupKFold, cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression, RidgeClassifier, BayesianRidge, Ridge, SGDClassifier, LinearRegression
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.model_selection import GroupShuffleSplit 
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import MinMaxScaler
from random import randint
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, RobustScaler, KBinsDiscretizer, MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline

In [ ]:
#Load the dataframe

df = pd.read_csv('MIMIC_WITHOUT_OUTLIERS_ITERATIVE_IMPUTATION.csv')

In [ ]:
#Divide the dataframe into train and test

train, test = train_test_split(df, test_size=0.2)

#Split the data into train and test based on GROUPID 

splitter = GroupShuffleSplit(test_size=.20, n_splits=2, random_state = 7)
split = splitter.split(df, groups=df['GROUP_ID'])
train_inds, test_inds = next(split)

train = df.iloc[train_inds]
test = df.iloc[test_inds]

In [ ]:
#Create X and Y for machine learning evaluation

X_train = train[['AGE', 'GENDER_M', 'MARITAL_STATUS_DIVORCED', 'MARITAL_STATUS_LIFE PARTNER',
       'MARITAL_STATUS_MARRIED', 'MARITAL_STATUS_SEPARATED',
       'MARITAL_STATUS_SINGLE', 'MARITAL_STATUS_UNKNOWN (DEFAULT)',
       'MARITAL_STATUS_WIDOWED', 'REL_DAY', 'BMI', 'HEART_RATE',
       'FLAG_HEART_RATE_ALARM_LOW', 'FLAG_HEART_RATE_ALARM_HIGH',
       'OXYGEN_SATURATION', 'FLAG_OXYGEN_SATURATION_ALARM_HIGH','FLAG_OXYGEN_SATURATION_ALARM_LOW', 
        'ARTERIAL_BLOOD_PRESSURE_SYSTOLIC', 'ARTERIAL_BLOOD_PRESSURE_DIASTOLIC']]

Y_train = train['DOD_LABEL']

testX = test[['AGE', 'GENDER_M', 'MARITAL_STATUS_DIVORCED', 'MARITAL_STATUS_LIFE PARTNER',
       'MARITAL_STATUS_MARRIED', 'MARITAL_STATUS_SEPARATED',
       'MARITAL_STATUS_SINGLE', 'MARITAL_STATUS_UNKNOWN (DEFAULT)',
       'MARITAL_STATUS_WIDOWED', 'REL_DAY', 'BMI', 'HEART_RATE',
       'FLAG_HEART_RATE_ALARM_LOW', 'FLAG_HEART_RATE_ALARM_HIGH',
       'OXYGEN_SATURATION', 'FLAG_OXYGEN_SATURATION_ALARM_HIGH','FLAG_OXYGEN_SATURATION_ALARM_LOW', 
        'ARTERIAL_BLOOD_PRESSURE_SYSTOLIC', 'ARTERIAL_BLOOD_PRESSURE_DIASTOLIC']]

testY = test['DOD_LABEL']

In [ ]:
pat_ids = df['GROUP_ID']

# define the scaler
scaler = MinMaxScaler()
# fit on the training dataset
scaler.fit(X_train)
# scale the training dataset
X_train = scaler.transform(X_train)

cv = list(GroupKFold(n_splits=4).split(X_train,Y_train,pat_ids))

In [ ]:
#Find the best hyperparameters

# Logistic Regression
from sklearn.linear_model import LogisticRegression
# define models and parameters
model = LogisticRegression()
parameters = {'C': np.logspace(-2, 0, 20),
              'penalty': ['none', 'l2', 'l1', 'elasticnet'],
              'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
              'multi_class': ['multinomial']}

# define grid search
grid_search = GridSearchCV(model, parameters, cv=cv, verbose=0, n_jobs=-1)
log_reg_cv = grid_search.fit(X_train, Y_train)
# summarize results
print("Tuned hpyerparameters (best parameters):", log_reg_cv.best_params_)

In [ ]:
# Ridge Classifier

from sklearn.linear_model import RidgeClassifier
model = RidgeClassifier()
alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# define grid search
grid = dict(alpha=alpha)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,error_score=0)
rig_clf_cv = grid_search.fit(X_train, Y_train)
# summarize results
print("Tuned hpyerparameters (best parameters):", rig_clf_cv.best_params_)

In [ ]:
# SVM

from sklearn.svm import SVC
model = SVC()
kernel = ['poly', 'rbf', 'sigmoid', 'linear']
C = [50, 10, 1.0, 0.1, 0.01]
gamma = ['scale']

# define grid search
grid = dict(kernel=kernel,C=C,gamma=gamma)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,error_score=0)
svm_cv = grid_search.fit(X_train, Y_train)

# summarize results
print("Tuned hpyerparameters (best parameters):", svm_cv.best_params_)

In [ ]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
parameters = {'criterion': ['gini', 'entropy', 'log_loss'],
              'max_depth': [2*n for n in range(1,10)],
              'max_features': ['auto', 'sqrt', 'log2'],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10]}

# define grid search
grid_search = GridSearchCV(model, parameters, n_jobs=-1, cv=cv,error_score=0)
rf_cv = grid_search.fit(X_train, Y_train)

# summarize results
print("Tuned hpyerparameters (best parameters):", rf_cv.best_params_)

In [ ]:
#KNN

from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
parameters = {'n_neighbors': list(range(1, 20)),
              'weights': ['uniform', 'distance'],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}

#define gid search
grid_search = GridSearchCV(model, parameters, n_jobs=-1, cv=cv,error_score=0)
knn_cv = grid_search.fit(X_train, Y_train)

# summarize results
print("Tuned hpyerparameters (best parameters):", knn_cv.best_params_)

In [ ]:
# XGBoost

from xgboost import XGBClassifier
param_grid = { 'max_depth': [4,5,6] , 'min_child_weight':[4,5,6] ,'learning_rate': [0.05,0.1,0.5] ,'n_estimators': [20,50,100] }
model=XGBClassifier()
grid = GridSearchCV(model, param_grid, cv=cv, n_jobs=-1)
#fit model to data
xgb_cv = grid.fit(X_train, Y_train)
print("Tuned hpyerparameters (best parameters):", xgb_cv.best_params_)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 9),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]
             ,"splitter":["best"]}
 
# Instantiating Decision Tree classifier
tree = DecisionTreeClassifier()
 
grid = GridSearchCV(model, param_grid, cv=cv, n_jobs=-1)
#fit model to data
xgb_cv = grid.fit(X_train, Y_train)
print("Tuned hpyerparameters (best parameters):", tree.best_params_)

In [ ]:
# After the best hyperparameters have been found, evaluation of Machine Learning models will be done to find best 
#AUROC score

model_names = ['Logistic Regression', 'Ridge Classifier', 'KNN', 'Random Forest', 'SVM', 'XGBoost', 'Decision Tree']
models = [LogisticRegression(C=0.03359818286283781, multi_class='multinomial',penalty='l2', solver='newton-cg'),
         RidgeClassifier(alpha=0.5), 
         KNeighborsClassifier(algorithm='auto', n_neighbors= 14, p=1, weights='uniform'),
         RandomForestClassifier(criterion='gini', max_depth=12, max_features='sqrt', min_samples_leaf=4, 
                                       min_samples_split=10),
         SVC(C=0.1, gamma='scale', kernel='linear'),
         XGBClassifier(learning_rate=0.1, max_depth=5, min_child_weight=4, n_estimators=20),
         DecisionTreeClassifier(criterion='entropy', max_depth=18, max_features='auto', min_samples_leaf=1, min_samples_split=2,
                              splitter='best')
         ]

print('Results of ad hoc modeling:')

##########################################Find AUROC charecteristics################

#print('\t\t\tX-val AUROC || train AUROC   ||   X-val AUPRC || train AUPRC')
#print('-----------------------------------------------------------------------------------------')

for i, model in enumerate(models):
    model_name = model_names[i]
    scores = cross_validate(model, X_train, Y_train, scoring=['roc_auc', 'average_precision'], 
                            cv=list(GroupKFold(n_splits=4).split(X_train,Y_train,pat_ids)), return_train_score=True, return_estimator=False, n_jobs=-1)
    print("Model Name: ", model_name)
    print('{0:<20s}: {1:.2f} +/- {2:.2f} || {3:.2f} +/- {4:.2f} || {5:.2f} +/- {6:.2f} || {7:.2f} +/- {8:.2f}'.format(model_name, 
                                                                       scores['test_roc_auc'].mean(), scores['test_roc_auc'].std(),
                                                                       scores['train_roc_auc'].mean(), scores['train_roc_auc'].std(), 
                                                                       scores['test_average_precision'].mean(), scores['test_average_precision'].std(),
                                                                       scores['train_average_precision'].mean(), scores['train_average_precision'].std())) 

In [ ]:
#After finding the AUROC scores for these models, AUROC plots will be drawn for these models

def draw_cv_roc_curve(classifier, cv, X_train, Y_train, title='ROC Curve'):
    # Creating ROC Curve with Cross Validation
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    i = 0
    for train, test in cv.split(X_train, Y_train,pat_ids):
        probas_ = classifier.fit(X_train.iloc[train], Y_train.iloc[train]).predict_proba(X_train.iloc[test])
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(y.iloc[test], probas_[:, 1])
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
        i += 1
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Luck', alpha=.8)
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    plt.plot(mean_fpr, mean_tpr, color='b',label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),lw=2, alpha=.8)
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,label=r'$\pm$ 1 std. dev.')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
#Logistic Regression

clf=LogisticRegression(C=0.03359818286283781, multi_class='multinomial', penalty='l2', solver='newton-cg')
# Set up Stratified K Fold
cv = GroupKFold(n_splits=4)
draw_cv_roc_curve(clf, cv, X_train, Y_train, title='Cross Validated LogisticRegression')

In [ ]:
#KNN

clf=KNeighborsClassifier(algorithm='auto', n_neighbors= 14, p=1, weights='uniform')
# Set up Stratified K Fold
cv = GroupKFold(n_splits=4)
draw_cv_roc_curve(clf, cv, X_train, Y_train, title='Cross Validated KNN')

In [ ]:
#RandomForest

clf=RandomForestClassifier(criterion='gini', max_depth=12, max_features='sqrt', min_samples_leaf=4, min_samples_split=10)
# Set up Stratified K Fold
cv = GroupKFold(n_splits=4)
draw_cv_roc_curve(clf, cv, X_train, Y_train, title='Cross Validated RandomForestClassifier')

In [ ]:
#XGBoost

clf=XGBClassifier(learning_rate=0.1, max_depth=5, min_child_weight=4, n_estimators=20)
# Set up Stratified K Fold
cv = GroupKFold(n_splits=4)
draw_cv_roc_curve(clf, cv, X_train, Y_train, title='Cross Validated XGBoost')

In [ ]:
#DecisionTree

clf=DecisionTreeClassifier(criterion='entropy', max_depth=18, max_features='auto', min_samples_leaf=1, 
                           min_samples_split=2, splitter='best')
# Set up Stratified K Fold
cv = GroupKFold(n_splits=4)
draw_cv_roc_curve(clf, cv, X_train, Y_train, title='Cross Validated DecisionTreeClassifier')

In [ ]:
#RidgeClassifier

clf = LogisticRegression(penalty='l2')
cv = GroupKFold(n_splits=4)
draw_cv_roc_curve(clf, cv, X_train, Y_train, title='Cross Validated RidgeClassifier')

In [ ]:
#SVM

clf=SVC(C=0.1, gamma='scale', kernel='linear', probability=True)
# Set up Stratified K Fold
cv = GroupKFold(n_splits=4)
draw_cv_roc_curve(clf, cv, X_train, Y_train, title='Cross Validated SVM')

In [ ]:
#Train the models

lr=LogisticRegression(C=0.03359818286283781, multi_class='multinomial',penalty='l2', solver='newton-cg').fit(X_train, Y_train)
rc=LogisticRegression(penalty='l2').fit(X_train, Y_train)
knn=KNeighborsClassifier(algorithm='auto', n_neighbors= 14, p=1, weights='uniform').fit(X_train, Y_train)
rf=RandomForestClassifier(criterion='gini', max_depth=12, max_features='sqrt', min_samples_leaf=4, 
                                       min_samples_split=10).fit(X_train, Y_train)
svm=SVC(C=0.1, gamma='scale', kernel='linear', probability=True).fit(X_train, Y_train)
xgb=XGBClassifier(learning_rate=0.1, max_depth=5, min_child_weight=4, n_estimators=20).fit(X_train, Y_train)
dt=DecisionTreeClassifier(criterion='entropy', max_depth=18, max_features='auto', min_samples_leaf=1, min_samples_split=2,
                              splitter='best').fit(X_train, Y_train)

In [ ]:
#Test the models using J statistics

model_names = ['Logistic Regression', 'Ridge Classifier', 'KNN', 'Random Forest', 'SVM', 'XGBoost', 'Decision Tree']

models = [lr, rc, knn, rf, svm, xgb, dt]

for i, model in enumerate(models):
    print('-------------------------------')
    model_name = model_names[i]
    print("Model Name is:", model_name)
    fpr, tpr, thresholds = roc_curve(testY, model.predict_proba(testX)[:, 1])
    # get the best threshold
    J = tpr - fpr
    ix = np.argmax(J)
    best_thresh = thresholds[ix]
    print('Best Threshold=%f' % (best_thresh))
    y_pred = (model.predict_proba(testX)[:, 1] > 0.008)
    ConfusionMatrixDisplay.from_predictions(testY, y_pred, normalize='true', cmap='Blues', display_labels = model.classes_)
    ConfusionMatrixDisplay.from_predictions(testY, y_pred, cmap='Blues', display_labels = model.classes_)
    print("Accuracy is:",accuracy_score(testY, y_pred))
    print("Precision is:",precision_score(testY, y_pred))
    print("Recall is:",recall_score(testY, y_pred))
    print("F1 is:",f1_score(testY, y_pred))